In [1]:
import torch


# Creating Layers

In [2]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases


# Activation Functions

Linear

In [4]:
class Activation_Linear:
    # Forward pass
    def forward(self, inputs):
        self.output = self.inputs


## Sigmoid

In [5]:
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))


In [6]:
class Loss_CategoricalCrossentropy() :
  # Forward pass
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss


In [7]:
softmax_outputs = torch.tensor([[0.7, 0.1, 0.2], [0.1, 0.5, 0.4],[0.02, 0.9, 0.08]])
class_targets = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0]])


In [8]:
x = Loss_CategoricalCrossentropy()
x.forward(softmax_outputs, class_targets)


tensor(1.6539)

In [13]:
X = torch.tensor([0.1, 0.5])
y = torch.tensor([0.05, 0.95])


In [15]:
hidden_layer_1 = DenseLayer(2, 2)
activation1 = Activation_Sigmoid()
output_layer = DenseLayer(2, 2)
activation2 = Activation_Linear()


In [16]:
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output


In [17]:
def back_prop(fp, X, y, output_layer, activation1, hidden_layer_1):
    lr = torch.tensor(0.01)

    # Output layer backpropagation with sigmoid activation
    back1 = (fp[0][0] - y[0]) * fp[0][0] * (1 - fp[0][0])
    back2 = (fp[0][1] - y[1]) * fp[0][1] * (1 - fp[0][1])

    output_layer.weights[0][0] -= lr * back1 * activation1.output[0][0]
    output_layer.weights[0][1] -= lr * back1 * activation1.output[0][1]
    output_layer.weights[1][0] -= lr * back2 * activation1.output[0][0]
    output_layer.weights[1][1] -= lr * back2 * activation1.output[0][1]
    output_layer.biases[0][0] -= lr * back1
    output_layer.biases[0][1] -= lr * back2

    # Hidden layer backpropagation with Linear activation
    hidden1_grad = (back1 * output_layer.weights[0][0] + back2 * output_layer.weights[0][1]) * (hidden_layer_1.output[0][0] > 0).float()
    hidden2_grad = (back1 * output_layer.weights[1][0] + back2 * output_layer.weights[1][1]) * (hidden_layer_1.output[0][1] > 0).float()

    hidden_layer_1.weights[0][0] -= lr * hidden1_grad * X[0]
    hidden_layer_1.weights[0][1] -= lr * hidden1_grad * X[1]
    hidden_layer_1.weights[1][0] -= lr * hidden2_grad * X[0]
    hidden_layer_1.weights[1][1] -= lr * hidden2_grad * X[1]
    hidden_layer_1.biases[0][0] -= lr * hidden1_grad
    hidden_layer_1.biases[0][1] -= lr * hidden2_grad


In [18]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)


In [19]:
loss = 0.0001


In [ ]:
y_pred = forward_pass(X)
err = error_calculation(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y)
